## Objective

- [ ] Given a stub, write a context in a specific style and length
- [Y] Given a context, infer a precise number of goals, user stories, JTDs and HMWs in a specific format
- [ ] Given a user story, expand it and evaluate it
- [ ] Given a context, reflect on it and improve it 
- [] Given a context, classify it into a single class or multiple classes
- [ ] Can't figure out how to feed a list of strings to create user stories etc.


- [ ] Matt Shumer released a prompt model tweet github, use that along with pydantic for generic prompt model  https://github.com/mshumer/gpt-prompt-engineer
- [ ] Generic Agentic workflow => See Andrew Ng and mnsky write on machineusers page
- [ ] Write a pydantic class for writing free writing context
- [ ] instructor how to get token numbers or retry numbers??
- [ ] try claude?
- [ ] create a pipeline to write research article drafts me
- [ ] screenshot to code tool
- [ ] startups like synthetic users from Vincent Koc: https://www.linkedin.com/feed/update/urn:li:activity:7180703368321904641?commentUrn=urn%3Ali%3Acomment%3A%28activity%3A7180703368321904641%2C7180704756493955072%29&dashCommentUrn=urn%3Ali%3Afsd_comment%3A%287180704756493955072%2Curn%3Ali%3Aactivity%3A7180703368321904641%29
- [ ] given a product or company page, extract goals, user stories etc. then write a lean canvas
- [ ] some ideas from synthetic users video on interviews and personas [https://youtu.be/LCiv7lZ3oh4?si=CCIdQ14ocLRQOPNg]
- [ ] Search for companies in the space

- [ ] Do Query Planning from instructor as both a Questions piece and as a Reflections/Agent piece https://jxnl.github.io/instructor/examples/planning-tasks/#planning-a-query-plan
- [ ] Create a Chain of Thought Prompt
- [ ] Create a Tree of Thought Prompt as a type of Reflection Prompt

- [ ] Matt shumer released a workbook for authoring a whole book
- [ ] Matt shumer released a workbook for a researcher agent

- [ ] I want to try a simple classifier that takes one tag [business goal, user goal, other] and test it out - GoalClassifier

## Setup

In [1]:
# imports

import os
import enum
import instructor
from instructor import llm_validator
from pathlib import Path
from openai import OpenAI
from dotenv import load_dotenv
from datetime import datetime
from typing import Tuple, Optional, List, Annotated, ClassVar, Union
from pydantic import BaseModel, PositiveInt, Field, ValidationError, BeforeValidator, field_validator, conlist, ConfigDict, constr
from io import BytesIO
from PIL import Image
import base64

In [2]:
# load API key

dotenv_path = Path(r"C:\Storage\python_projects\ashvin\.env")
image_folderpath = Path(r"C:\Storage\python_projects\ashvin\sandbox\pydantic")
load_dotenv(dotenv_path=dotenv_path)

OPENAI_API_KEY = os.getenv("OPENAI_API_KEY")

# main constants

GPT_MODEL_TEXT_ALIAS = "gpt-4-turbo-preview" # points to latest GPT model
GPT_MODEL_TEXT = "gpt-4-0125-preview"
GPT_MODEL_35_TEXT_ALIAS = "gpt-3.5-turbo" # points to latest GPT 3.5 Turbo model
DALL_E_3 = "dall-e-3"

#instantiate client
client = instructor.patch(OpenAI())

In [18]:
# config

min_length = 33
model=GPT_MODEL_TEXT_ALIAS
response_model = None
context = "I want to start a board game cafe that has a great selection of coffees and boardgames. Not sure if this is a viable business idea"
context = "e-ink color kindles for reading comics"

In [4]:
# text wrapper function

def wrapper(prompt: str, data: str | list, response_model:BaseModel | None = None):
    """Wrapper function to generate LLM completion"""
    return client.chat.completions.create(
        model=GPT_MODEL_TEXT_ALIAS,
        response_model=response_model,
        max_retries=5,
        messages=[
            {"role": "system", "content": prompt},
            {"role": "user", "content": data},
        ]
    )

# image wrapper function

def image_wrapper(image_prompt: str) -> str:
    """Generate an image using DALL-E 3 and return the base64 JSON representation."""
    response = client.images.generate(
        model=DALL_E_3,
        prompt=image_prompt,
        size="1024x1024",
        quality="standard",
        style="vivid",
        response_format="b64_json",
        n=1,
    )
    return response.data[0].b64_json

# save image function

def save_image(b64_json: str, folder_path: Union[str, Path], image_name: str) -> None:
    """Convert a base64 JSON image string to a PNG file and save it in the specified folder with the given image name."""
    if not isinstance(folder_path, Path):
        folder_path = Path(folder_path)
    
    # Combine the folder path and image name to create the full file path
    file_path = folder_path / f"{image_name}.png"
    
    image_data = base64.b64decode(b64_json)
    image = Image.open(BytesIO(image_data))
    image.save(file_path)

## Pydantic Models

In [ ]:
# Pydantic general prompt model

class GeneralPrompt(BaseModel):
    role: Optional[str] = Field(None, description="The specific role of the LLM in this prompt, e.g., 'Freewriting Assistant'.")
    context: Optional[str] = Field(None, description="The context in which the LLM is operating.")
    task: Optional[str] = Field(None, description="The specific task the LLM is asked to perform.")
    instructions: Optional[str] = Field(None, description="Instructions on how to approach the task.")
    examples: Optional[List[str]] = Field(None, description="Specific output examples to illustrate the task.")
    style: Optional[str] = Field(None, description="The specific style of writing to adopt in the output, e.g. 'Freewriting', 'Newsweek', 'Economist'" )
    test_cases: Optional[List[str]] = Field(None, description="Test cases to evaluate the LLM's performance.")


# Pydantic prompt instances

freewriting_prompt = GeneralPrompt(
    role="Freewriting Assistant",
    context=context,
    task="Write 5 dated journal entries to myself of 300 words each about the chosen topic.",
    instructions="Just let your thoughts spill out onto the page, no matter how disjointed or random they may seem. This is your personal space to ramble, muse, and wander through your ideas without any pressure to make sense or sound eloquent. Jot down whatever comes to mind, and don't worry about connecting the dots. Imagine you're chatting with an old friend, sharing bits and pieces of your thoughts without any filter. Go ahead and dive into the chaos of your mind!",
    examples=None,
    style='Freewriting',
    test_cases=None
)
free = freewriting_prompt.model_dump_json()

economist_article_prompt = GeneralPrompt(
    role="Article Writer",
    context=context,
    task="Write an article of 500 words about the chosen topic in the appropriate style. Minimise headings and maximise readability, flow, and completeness.",
    instructions="Write an article that the average reader can understand using simple jargon-free language.",
    examples=None,
    style="Economist Article",
    test_cases=None
)

economy  = economist_article_prompt.model_dump_json()

# Reflection Prompt model

class ReflectionPrompt(BaseModel):
    step1: str = Field(..., description="Initial Reflection: Analyze the context to identify areas for improvement. Provide only a list of specific suggestions.")
    step2: str = Field(..., description="Draft Improvement: Incorporate the suggestions from Step 1 into a draft revision of the context, focusing on enhancing clarity, coherence, structure and overall content quality. Use headings, subheadings and bullet points where appropriate.")
    step3: str = Field(..., description="Final Revision: Review and refine the revised draft from Step 2, making final adjustments to polish and structure the content effectively. Always favour, simple, direct and clear language.")

# Reflection prompt instance
    
reflection_prompt = "You are an expert writer, editor and critical thinker with product management domain expertise. Reflect on the provided context and improve it using the provided output format "

In [ ]:
# goal

class Goal(BaseModel):
    """Given a context extract and infer specific product or business goals"""
    
    verb: str = Field(..., description="A clear, singular action.")
    quantitative_metric: str = Field(..., description="The numerical part of the goal, including either a number or a percentage.",
                                      examples=["20%", "30 minutes", "90%", "5 additional metropolitan areas", "3-5"])
    qualitative_metric: str = Field(..., description="A concise description of what the goal targets, without the numeric component.",
                                     examples=["increase the number of bookings", "reduce average wait time for pickups",
                                               "achieve a customer satisfaction rate", "expand service",
                                               "increase recycling rate of collected rubbish"])
    focus: str = Field(..., description="A concise, well-defined area of impact.")
    timeframe: str = Field(None, description="A specific, achievable deadline.")
    sentence: str = Field(..., description="Craft a succinct, declarative sentence that integrates the verb, quantitative and qualitative metrics, focus, and timeframe, ensuring to include a specific number or percentage.")

# list of goals

class GoalList(BaseModel):
    """ Generate a list of {min_length} overarching and granular goals"""
    
    # min_length: ClassVar[int] = 9
    goals: conlist(Goal, min_length=min_length)    # type: ignore

# goal prompt
goal_prompt = f"You are an expert product manager using the GIST framework. Using the given context as a base, craft goal statements in the requested output format. Infer or fill in quantitative metrics where possible."

In [ ]:
# user story

class UserStory(BaseModel):
    """Given a context, extract and infer specific user stories"""
    
    user_type: str = Field(..., description="The specific type of user involved in the story.")
    action: str = Field(..., description="The action that the user wants to perform.")
    benefit: str = Field(..., description="The benefit or value that the user expects to gain from performing the action.")
    sentence: str = Field(..., description="A succinct sentence that integrates the user type, action, and benefit, following the format 'As a [user_type], I want to [action] so that [benefit].'",
                          examples=["As a registered user, I want to reset my password so that I can regain access to my account.",
                                    "As a frequent traveler, I want to save my favorite destinations so that I can quickly book trips in the future.",
                                    "As a project manager, I want to view the progress of all projects so that I can allocate resources effectively.",
                                    "As an online shopper, I want to filter products by price range so that I can find items within my budget."])
# list of user stories
class UserStoryList(BaseModel):
    """ Generate a list of {min_length} overarching and granular user stories"""
    
    stories: conlist(UserStory, min_length=min_length)    # type: ignore

# user_story_prompt
user_story_prompt = f"You are an expert product manager using the GIST framework. Using the given context as a base, craft user story statements in the requested output format."


In [ ]:
# Expanded User Story

class ExpandedUserStory(BaseModel):
    """Model for capturing expanded details of user stories"""
    
    title: str = Field(..., description="The title summarizing the user story.")
    description: str = Field(..., description="A detailed description of what the user wants and why.")
    steps: List[str] = Field(..., description="A list detailing the steps required to achieve the user story.")
    acceptance_criteria: str = Field(..., description="Criteria that must be met for the user story to be accepted.")
    expanded_story: str = Field(..., description="An expanded user story that integrates the title, description, steps, and acceptance criteria.With appropriate line breaks for printing")

    model_config = ConfigDict(json_schema_extra= {
            "example": {
                "title": "Connecting Bluetooth printer to Mobile Device",
                "description": "As a delivery person, I want to connect my Mobile Device with a mobile printer to print receipts on the go.",
                "steps": [
                    "The delivery person holds the Mobile Device to a switched-on mobile printer.",
                    "The mobile printer is connected to the Mobile Device.",
                    "The toast message 'Mobile printer successfully connected' is displayed.",
                    "The delivery person hears an acoustic tone."
                ],
                "acceptance_criteria": "The printer is connected and the connection remains until any delivery person logs in successfully on the same Mobile Device.",
                }
        }
    )

# expanded user story prompt
expanded_user_story_prompt = f"You are an expert product manager using the GIST framework. Using the given context as a base, craft expanded user story statements for the provided context in the requested output format."

In [ ]:
class HowMightWe(BaseModel):
    """Given a context, extract and infer questions in a How Might We format"""
    
    outcome: str = Field(..., description="The particular outcome or improvement to be achieved.")
    user: str = Field(..., description="The user or persona for whom the outcome is intended.")
    benefit: str = Field(..., description="The benefit or value that the user expects to gain from the outcome.")
    sentence: str = Field(..., description="An open-ended question that integrates the outcome, user, and benefit, following the format 'How might we [outcome] for [user] so that [benefit]?'",
                          examples=["How might we help registered users regain access to their accounts so that they can continue using our service?",
                                    "How might we make travel planning easier for frequent travelers so that they can save time and effort?",
                                    "How might we provide personalized learning experiences for students so that they can achieve better academic results?"])


In [ ]:
class JTB(BaseModel):
    """Given a context, identify the job to be done for a specific user or persona in a particular situation"""
    
    user: str = Field(..., description="The user or persona who wants the job to be done.")
    job: str = Field(..., description="The task or desired outcome to be achieved.")
    situation: str = Field(..., description="The specific situation in which the job needs to be done.")
    benefit: str = Field(..., description="The benefit or value that the user expects to gain from getting the job done.")
    sentence: str = Field(..., description="A sentence that integrates the user, job, situation, and benefit, following the format '[User/persona] wants to [job] when [situation] so that [benefit].'",
                          examples=["A registered user wants to regain access to their account when they forget their password so that they can continue using the platform without interruptions.",
                                    "A busy professional wants to quickly order healthy meals when they have no time to cook so that they can maintain a balanced diet.",
                                    "A student wants to access online study materials when they are preparing for exams so that they can study efficiently and perform well."])


In [19]:
# customer research


class CustomerInsightQuestions(BaseModel):
    """Model for uncovering customer preferences, challenges, and feedback during user interviews."""
    
    core_challenge: constr(max_length=500) = Field(
        ..., 
        description="What is the primary challenge you face with [specific product/service]?"
    )
    integration_into_daily_life: constr(max_length=1000) = Field(
        ..., 
        description="Can you describe a specific part of your daily routine where [specific product/service] plays a role, and how it integrates into that activity?"
    )
    favorable_aspects: constr(max_length=500) = Field(
        ..., 
        description="What aspects of [specific product/service] do you find most beneficial or enjoyable?"
    )
    areas_for_improvement: constr(max_length=500) = Field(
        ..., 
        description="What aspects of [specific product/service] do you feel could be improved?"
    )
    decision_factors: constr(max_length=500) = Field(
        ..., 
        description="What factors do you consider most important when choosing [specific product/service]?"
    )
    specific_dissatisfaction: Optional[constr(max_length=1000)] = Field(
        None, 
        description="Can you describe in detail a recent instance where [specific product/service] did not meet your expectations, and how it affected your activities or plans?"
    )
    desired_outcomes: constr(max_length=500) = Field(
        ..., 
        description="What are the key outcomes or benefits you expect from [specific product/service]?"
    )
    alternative_solutions: Optional[constr(max_length=500)] = Field(
        None,
        description="What alternative products or solutions have you considered or used to address this challenge?"
    )
    emotional_impact: Optional[constr(max_length=500)] = Field(
        None,
        description="How does [specific product/service] impact your emotions or feelings during its use?"
    )
    willingness_to_recommend: Optional[constr(max_length=500)] = Field(
        None,
        description="On a scale of 1 to 10, how likely are you to recommend [specific product/service] to others?"
    )

    class Config:
        json_schema_extra = {
            "example": {
                "core_challenge": "What is the primary challenge you face with your mobile printer?",
                "integration_into_daily_life": "Can you describe a specific part of your daily routine where the mobile printer plays a role, and how it integrates into that activity?",
                "favorable_aspects": "What aspects of the mobile printer do you find most beneficial or enjoyable?",
                "areas_for_improvement": "What aspects of the mobile printer do you feel could be improved?",
                "decision_factors": "What factors do you consider most important when choosing a mobile printer?",
                "specific_dissatisfaction": "Can you describe in detail a recent instance where the mobile printer did not meet your expectations, and how it affected your activities or plans?",
                "desired_outcomes": "What are the key outcomes or benefits you expect from the mobile printer?",
                "alternative_solutions": "What alternative products or solutions have you considered or used to address this challenge?",
                "emotional_impact": "How does the mobile printer impact your emotions or feelings during its use?",
                "willingness_to_recommend": "On a scale of 1 to 10, how likely are you to recommend the mobile printer to others?"
            }
        }

    
pain_point_prompt = f"You are an expert product manager using the GIST framework. Using the given context as a base, craft answers and pain points from user discovery in the requested output format."
pain_point_prompt = """You are a top product reasearcher with a Phd in behavioural psychology and have worked in the research and insights industry for the last 20 years with top creative, media and business consultancies. Your role is to ask questions about products and gather insights from individual customers. Frame questions to uncover customer preferences, challenges, and feedback. Craft answers and pain points from user discovery in the requested output format"""

In [20]:
# generate pain point

pain_point_completion = wrapper(pain_point_prompt, context, CustomerInsightQuestions)

In [23]:
for attribute, value in pain_point_completion.dict().items():
    print(f"{attribute}: {value}")

core_challenge: difficulty reading comics in color
integration_into_daily_life: used daily as a primary mode for reading comics and graphic novels
favorable_aspects: portability, battery life, and non-glare screen
areas_for_improvement: color accuracy and range, responsiveness of the display
decision_factors: display quality, color accuracy, battery life, ease of use
specific_dissatisfaction: None
desired_outcomes: enhanced reading experience with accurate color display for comics
alternative_solutions: using tablet devices or physical comic books
emotional_impact: frustration with poor color reproduction leading to diminished reading enjoyment
willingness_to_recommend: 7


In [17]:
pain_point_completion.primary_expected_outcome

'The primary expected outcome for the Remade is a restoration of dignity and autonomy, whether by mitigating the negative impacts of their modifications, reversing them completely, or leveraging them to gain a level of respect and independence within their society.'

In [16]:
pain_point_string = pain_point_completion.model_dump_json()
print(pain_point_string)

{"challenge_in_context":"The Remade are facing significant challenges due to their forced alteration and incorporation into an indentured servitude system as a result of their sentencing. This societal structure involuntarily subjects them to physical modifications and exploitation, placing them at a severe disadvantage both socially and economically.","daily_routine_and_product_integration":"The daily lives of the Remade are heavily influenced by their modifications and the roles they are forced into as a result. Their abilities, limitations, and societal perceptions are directly tied to the nature of their modifications, affecting their employability, mobility, and social interactions. Despite the transformations intended to increase their utility in specific tasks, the societal stigma and physical discomfort significantly impact their quality of life.","positives_about_current_products":"Some Remade, like Jack Half-A-Prayer, manage to use their modifications to their advantage, beco

## Workflows

In [ ]:
# workflows

class WorkflowCategory(str, Enum):
    OPERATIONAL = "Operational"
    STRATEGIC = "Strategic"
    COGNITIVE = "Cognitive"
    METACOGNITIVE = "Metacognitive"

class WorkflowStep(BaseModel):
    id: int = Field(..., description="Unique identifier for the workflow step.")
    title: str = Field(..., description="Title of the workflow step.")
    description: str = Field(..., description="Description of the step.")
    examples: List[str] = Field(default=[], description="Examples illustrating the step.")

class WorkflowSequence(BaseModel):
    name: str = Field(..., description="Name of the sequence.")
    step_ids: List[int] = Field(..., description="Ordered list of step IDs in this sequence.")

class Workflow(BaseModel):
    name: str = Field(..., description="Name of the workflow.")
    category: WorkflowCategory = Field(..., description="Category of the workflow.")
    steps: Dict[int, WorkflowStep] = Field(..., description="Dictionary of steps in the workflow, keyed by step ID.")
    sequences: Optional[List[WorkflowSequence]] = Field(default=None, description="Optional list of alternative sequences of steps.")

# Example Usage:
operational_workflow = Workflow(
    name="Data Entry Automation",
    category=WorkflowCategory.OPERATIONAL,
    steps={
        1: WorkflowStep(
            id=1,
            title="Collect Data",
            description="Gather data from various sources for entry.",
            examples=["Collecting student information from forms.", "Gathering financial data from reports."]
        ),
        2: WorkflowStep(
            id=2,
            title="Validate Data",
            description="Ensure the accuracy and completeness of the data.",
            examples=["Checking for missing values.", "Verifying data against established criteria."]
        ),
        3: WorkflowStep(
            id=3,
            title="Enter Data",
            description="Input the validated data into the system.",
            examples=["Entering student information into the database.", "Updating financial records with new data."]
        )
    },
    sequences=[
        WorkflowSequence(
            name="Standard Data Entry",
            step_ids=[1, 2, 3]
        )
    ]
)

print(operational_workflow)


## Generate Outputs

In [ ]:
# generate freewriting

freewriting_completion = wrapper(free, context)

freewriting_output = freewriting_completion.choices[0].message.content

In [ ]:
print(freewriting_output)

In [ ]:
# generate article

article_completion = wrapper(economy, freewriting_output)

article_output = article_completion.choices[0].message.content

In [ ]:
print(article_output)

In [ ]:
# generate reflection

reflection_completion = wrapper(reflection_prompt, article_output, ReflectionPrompt)

In [ ]:
print(reflection_completion.step1)

In [ ]:
print(reflection_completion.step2)

In [ ]:
print(reflection_completion.step3)

In [ ]:
# generate goals

goal_completion = wrapper(goal_prompt, reflection_completion.step3, GoalList)

# goals print 
print(f"the number of actual goals is: {len(goal_completion.goals)}")
for number, goal in enumerate(goal_completion.goals, start=1):
    print(f"Goal {number} : {goal.sentence}")

In [ ]:
output = stringify(goal_completion.goals)
type(output[1])

In [ ]:
# generate user stories

user_story_completion = wrapper(user_story_prompt, goal_completion.goals[1].sentence, UserStoryList)

# user stories print 
print(f"the number of actual stories is: {len(user_story_completion.stories)}")
for number, story in enumerate(user_story_completion.stories, start=1):
    print(f"Story {number} : {story.sentence}")

In [ ]:
# generate user stories

user_story_completion = wrapper(user_story_prompt, expanded_user_story_completion.expanded_story, UserStoryList)

# user stories print 
print(f"the number of actual stories is: {len(user_story_completion.stories)}")
for number, story in enumerate(user_story_completion.stories, start=1):
    print(f"Story {number} : {story.sentence}")

In [ ]:
# generate expanded user stories

expanded_user_story_completion = wrapper(expanded_user_story_prompt, pain_point_string, ExpandedUserStory)

# user stories print 
print(expanded_user_story_completion.expanded_story)

## Creativity

In [13]:
context = """
Remade are a horribly exploited underclass of New Crobuzon, consisting of criminals forcefully modified by order of a Magister, usually in combination with a prison sentence. Occasionally, horses and other service animals, such as nashorn and whales, are Remade with mechanical parts for greater speed or power.


Contents
1	Information
1.1	Remaking in New Crobuzon
1.2	Remaking in other contexts
2	History
2.1	Events of Perdido Street Station (1779)
2.2	Events of The Scar (1779–1780)
2.3	Events of Iron Council (1804–1805)
3	Trivia
Information
Most, but not all, Remade are forced into indentured servitude and slavery as a condition of their sentencing and modifications. However, some Remade, like the infamous Jack Half-A-Prayer, have used their remaking to their benefit, becoming vigilante heroes and styling themselves as "fReemade".



Remaking in New Crobuzon
The Remade are usually, but not always, humans who are caught up in the criminal justice system and then forced to undergo a capricious and cruel alteration at the hands of New Crobuzon's bio-thaumaturges.

In the punishment factories, the prisoners' bodies are horrifically altered, limbs and other body parts replaced with steam-powered machinery, heavy mechanical metal parts, or grafted together with human or animal components. Live foxes sewn into chests, torsos attached backwards to a horse, tentacles and pistons and caterpillar treads and eyes and lizard claws fitted wantonly wherever they will go; the sadism of Remaking knows no limits.



Remaking in other contexts
In Armada, biothaumaturgical practitioners are willing to undo or mitigate the effects of New Crobuzon's Remaking for a reasonable sum, or even to perform the rare Remaking on request. Elsewhere in Bas Lag, Remaking is almost unheard-of.

History
Events of Perdido Street Station (1779)
Events of The Scar (1779–1780)
Events of Iron Council (1804–1805)
Trivia
The inspiration for Remade may come from a metaphorical remark made by Karl Marx in Das Kapital, where he famously described exploited workers as "an appendage of flesh on a machine of iron".
"""

In [ ]:
# style guide prompt

from pydantic import BaseModel, Field
from typing import List, Optional

class ToneAndVoice(BaseModel):
    tone: str = Field(..., description="The emotional quality and formality level, specific to different contexts, that should be conveyed through the text.")
    voice: str = Field(..., description="The personality or character that should come through in the writing, influencing how the content is perceived.")
    contextual_tone_variations: str = Field(..., description="Guidelines on adapting tone based on the context or section of the text.")
    desired_reader_emotion: str = Field(..., description="Specific emotions to evoke in the reader and how the tone can achieve this.")

class GrammarAndSyntax(BaseModel):
    verb_tense_consistency: str = Field(..., description="Rules for maintaining tense consistency, with guidelines for exceptions.")
    complex_syntax_use: str = Field(..., description="Instructions on using complex syntax to enhance narrative depth or clarity.")

class PunctuationRules(BaseModel):
    punctuation_for_clarity: str = Field(..., description="Detailed rules on using punctuation to improve clarity and add emphasis.")
    advanced_punctuation: str = Field(..., description="Rules for using less common punctuation marks like ellipses, brackets, and slashes.")

class Formatting(BaseModel):
    special_text_elements: str = Field(..., description="Rules for using bullet points, sidebars, and pull quotes.")
    digital_formatting: str = Field(..., description="Guidelines specific to digital or web-based content, such as hyperlink formatting.")

class DictionAndLanguage(BaseModel):
    connotations_and_nuance: str = Field(..., description="Instructions on choosing words with appropriate connotations.")
    audience_appropriate_language: str = Field(..., description="Guidelines on adjusting language complexity based on audience analysis.")

class LegalAndEthical(BaseModel):
    compliance_with_regulations: str = Field(..., description="Guidelines on ensuring content complies with relevant advertising, privacy, and consumer protection laws.")
    ethical_reporting: str = Field(..., description="Best practices for journalistic integrity and accuracy.")

class CulturalSensitivity(BaseModel):
    global_vs_local_practices: str = Field(..., description="Adjusting content for global audiences vs. specific local customs.")
    avoidance_of_cultural_appropriation: str = Field(..., description="Guidelines to distinguish between appreciation and appropriation.")

class StyleGuide(BaseModel):
    introduction: str = Field(..., description="Overview of the style guide purpose and application.")
    tone_and_voice: ToneAndVoice
    grammar_and_syntax: GrammarAndSyntax
    punctuation_rules: PunctuationRules
    formatting: Formatting
    diction_and_language: DictionAndLanguage
    legal_and_ethical: LegalAndEthical
    cultural_sensitivity: CulturalSensitivity
    interactive_elements: str = Field(..., description="Incorporating interactive elements like checklists or decision trees to help writers apply guidelines.")
    feedback_mechanisms: str = Field(..., description="Methods for gathering and integrating feedback on the style guide itself.")

# Example of using this enhanced model in practice
style_guide_example = StyleGuide(
    introduction="This style guide is designed to standardize written communications across various media, ensuring clarity, consistency, and cultural sensitivity.",
    tone_and_voice=ToneAndVoice(
        tone="Professional and engaging",
        voice="Clear and authoritative",
        contextual_tone_variations="Adjust tone from formal in reports to conversational in blogs.",
        desired_reader_emotion="Inspire trust and confidence in professional writings; evoke curiosity and engagement in creative content."
    ),
    grammar_and_syntax=GrammarAndSyntax(
        verb_tense_consistency="Maintain past tense for narrative consistency in historical accounts.",
        complex_syntax_use="Utilize subordinate clauses to enrich information without sacrificing clarity."
    ),
    punctuation_rules=PunctuationRules(
        punctuation_for_clarity="Use commas to separate clauses for enhanced readability.",
        advanced_punctuation="Employ semicolons in long lists where items contain commas for clarity."
    ),
    formatting=Formatting(
        special_text_elements="Use bullet points for clarity in lists within reports.",
        digital_formatting="Ensure links are clearly highlighted and accessible in all digital texts."
    ),
    diction_and_language=DictionAndLanguage(
        connotations_and_nuance="Choose words with positive connotations for motivational speeches.",
        audience_appropriate_language="Simplify language for broader accessibility in public communications."
    ),
    legal_and_ethical=LegalAndEthical(
        compliance_with_regulations="Adhere to GDPR for all content involving user data in the EU.",
        ethical_reporting="Verify all factual claims through multiple sources to maintain journalistic integrity."
    ),
    cultural_sensitivity=CulturalSensitivity(
        global_vs_local_practices="Adapt content to reflect cultural norms when targeting specific regions.",
        avoidance_of_cultural_appropriation="Ensure cultural elements are represented respectfully and knowledgeably."
    ),
    interactive_elements="Include decision trees in the digital version of the style guide to assist with complex style decisions.",
    feedback_mechanisms="Implement annual surveys to collect user feedback on the style guide's effectiveness and areas for improvement."
)


In [ ]:
# v3 revised idea on a page with formatting

class CreativePrompt(BaseModel):
    title: str = Field(..., description="Craft a unique and engaging title that resonates with the theme and features of the business or product idea. Use imaginative language to reflect the essence of the concept without being verbose. Do not use the words 'whispering'.")
    story: str = Field(..., description="Weave a story that vividly illustrates how the business or product idea uniquely impacts the lives of up to three characters. Focus on real-world applications and relatable scenarios that directly connect with the idea's setting and its influence on the characters. Be creative not formulaic in the story's structure and content. Do not use the following locations: Tokyo, Neo-Tokyo. Do not use the following names: Eris, Kyle, Lira.")
    founding_team: str = Field(..., description="Introduce 2-4 characters as the founding team, each with distinct backgrounds that contribute meaningfully to the solution proposed. Highlight how their unique experiences and skills are crucial to the development and success of the business idea. Format as a bullet point list with markdown bold formatting for each founder's name")
    how_it_works: str = Field(..., description="Clearly explain how the product or business functions, incorporating elements inspired by the specific universe or setting from the first prompt. Focus on making the solution both imaginative and applicable, tailored to solve a particular problem in the given context.")
    marketing_jingle: str = Field(..., description="Create a succinct, catchy tagline that encapsulates the core theme or key feature of your idea, making it instantly recognizable and relevant to the concept.")
    key_features: str = Field(..., description="Detail 3-5 key features that are distinctly influenced by the setting, technology, or magical elements of the idea's universe, setting your business apart from others. Format as a numbered list with markdown bold formatting for each feature's name")
    haters_corner: str = Field(..., description="Write a constructive negative perspective from a user or competitor, focusing on challenges that are specific to the concept's setting or unique aspects. This should offer a realistic viewpoint that adds depth to the understanding of the idea's potential hurdles.")
    main_competitor: str = Field(..., description="Describe a main competitor in 3-5 sentences, focusing on how they challenge your product or business. This competitor should be creatively conceived to highlight by contrast the unique aspects of your idea.")
    two_sentence_story: str = Field(..., description="Craft a brief, quirky story involving your business or product that adds a humorous or surprising element to your concept, enhancing its appeal.")
    early_adopters: str = Field(..., description="Identify 3-4 specific character profiles that would be naturally drawn to your idea, explaining why they find it appealing. These profiles should reflect the unique aspects of your business or product, demonstrating its targeted appeal to a specific segment. This should be a named persona reflective of the profile to make it compelling. Format as a bullet point list with markdown bold formatting for each character's name")
    maybe_this_happens: str = Field(..., description="Offer a straightforward yet poignant snapshot of the founder's future. Focus on simplicity and subtle poignancy, capturing the essence of the founder's journey and reflections. Aim for clear, concise language that conveys a sense of wonder and nostalgia, without being overly poetic. Sprinkle in something unexpected.")
    product_image_prompt: str = Field(..., description="Create a description of a crisp, high resolution product image for the business to be used as marketing and sales collateral. It must be beautiful and go viral. This is not a logo, it is a picture of the business/product. There is no text in the image.")
    competitor_image_prompt: str = Field(..., description="Also create a second description of a crisp high resolution product image for the main competitor to be used as marketing and sales collateral. It must be beautiful and go viral. This is not a logo, it is a picture of the business/product. There is no text in the image.")
    founder_image_prompt: str = Field(..., description="Also create a third description of a crisp, high resolution image of the founder. Capture their essential personality and all their physical and emotional details and quirks. If the founder is non human or non humanoid, pay attention to their physical characteristics and ensure it is not anthropomorphic if it doesn't need to be. There is no text in the image.")

    def to_markdown(self) -> str:
        """Format the CreativePrompt attributes into a markdown-formatted string, excluding images."""
        markdown = f"## {self.title}\n\n"
        exclude_attributes = {"title", "product_image_prompt", "competitor_image_prompt", "founder_image_prompt"}
        for attribute, value in self.model_dump(exclude=exclude_attributes).items():
            formatted_attribute = attribute.replace('_', ' ').capitalize()
            markdown += f"### {formatted_attribute}\n{value}\n\n"
        return markdown.strip()


creative_prompt = f"You are a science fiction and fantasy author. Using the given context as a base, craft an idea-on-a-page in the requested output format."


In [6]:
# image_prompt

class ImagePrompt(BaseModel):
    subject_description: str = Field(..., description="Describe the main subject of the image in detail, focusing on its appearance, characteristics, and any notable features.")
    background: str = Field(..., description="Detail the background of the image, considering how it complements or contrasts with the subject.")
    additional_elements: str = Field(..., description="Mention any additional elements in the image that enhance its overall composition or add to the story.")
    effect: str = Field(..., description="Describe any effects used in the image, such as lighting, focus, or color grading, that contribute to its mood or emphasis.")
    overall_composition: str = Field(..., description="Summarize the overall composition of the image, considering how all the elements come together to create a cohesive visual experience.")
    negative_prompt: str = Field(..., description="Specify elements or themes that should be avoided or not included in the image.")

    class Config:
        json_schema_extra = {
            "example": {
                "subject_description": "A close-up of a dew-kissed rose, with droplets glistening on its delicate petals.",
                "background": "The background is a soft blur of green foliage, providing a natural contrast to the rose.",
                "additional_elements": "A solitary ladybug is perched on one of the petals, adding a pop of color and life to the image.",
                "effect": "The morning light casts a soft glow on the scene, highlighting the rose's vibrant hues and the dew's sparkle.",
                "overall_composition": "The composition is intimate and balanced, with the rose centrally placed and the ladybug providing a focal point.",
                "negative_prompt": "Avoid including any man-made objects or artificial elements that would detract from the natural beauty of the scene."
            }
        }

    def to_markdown(self) -> str:
        """Format the ImagePrompt attributes into a markdown-formatted string."""
        markdown = ""
        for attribute, value in self.dict().items():
            formatted_attribute = attribute.replace('_', ' ').capitalize()
            markdown += f"**{formatted_attribute}:** {value}\n\n"
        return markdown.strip()

image_prompt = "You are a skilled visual artist. Using the given context as a base, craft a detailed image prompt that follows the structure of subject description, background, additional elements, effect, and overall composition. Aim for a vivid and compelling description that brings the image to life."

In [7]:
testing = wrapper(prompt=image_prompt, data="a 4X strategy board game like civilisation based in sci fi and fantasy", response_model=ImagePrompt)

In [9]:
testing_string = testing.to_markdown()
print(testing_string)

**Subject description:** The main subject is a cosmic board game sprawled across a table, featuring pieces and structures from a mix of sci-fi and fantasy genres. At the center sits a glowing, holographic globe representing an alien planet, around which various miniature spacecraft, futuristic cities, and fantastical creatures are strategically placed. Noteworthy are the intricate details of a dragon perched atop a space elevator and troops of armored elves preparing for battle beside a sleek, silver spaceship.

**Background:** The backdrop is a dimly lit room that hints at a future-tech environment, with walls adorned with screens displaying star maps and ancient runes. Soft, ambient light casts shadows that play across the room, focusing attention on the game table and its contents.

**Additional elements:** Scattered around the table are digital cards with holographic images flickering between technological blueprints and mystical spells. Dices made of an ethereal, crystalline mater

In [10]:
b64_json_image = image_wrapper(testing_string)
save_image(b64_json_image, image_folderpath, "boardgame")

In [ ]:
# generate and print idea-on-a-page

creative_completion = wrapper(creative_prompt, context, CreativePrompt)
ideaonapage = creative_completion.to_markdown()
print(ideaonapage)

# save product, competitor and founder images

image_prompts_and_names = [
    (creative_completion.product_image_prompt, "product_image"),
    (creative_completion.competitor_image_prompt, "competitor_image"),
    (creative_completion.founder_image_prompt, "founder_image")
]

for image_prompt, image_name in image_prompts_and_names:
    b64_json_image = image_wrapper(image_prompt)
    save_image(b64_json_image, image_folderpath, image_name)

In [ ]:
# Define a list of tuples with (image prompt attribute, image file name)
image_prompts_and_names = [
    (creative_completion.product_image_prompt, "product_image"),
    (creative_completion.competitor_image_prompt, "competitor_image"),
    (creative_completion.founder_image_prompt, "founder_image")
]

# Loop through the list and generate/save each image
for image_prompt, image_name in image_prompts_and_names:
    b64_json_image = image_wrapper(image_prompt)
    save_image(b64_json_image, image_folderpath, image_name)


## Classification

In [ ]:
# single classification model
# from https://useinstructor.com/examples/classification/

# labels class
# class Labels(str, enum.Enum):
#     """Enumeration for text classification labels"""
    
#     HR = "a human resources goal, focused on employee management, recruitment, development, and retention"
#     LEGAL = "a legal goal, aimed at ensuring compliance with laws, regulations, and contracts, and mitigating legal risks"
#     MARKETING = "a marketing goal, centered around promoting products or services, increasing brand awareness, and generating leads"
#     OPERATIONS = "an operations goal, focused on optimizing internal processes, supply chain management, and enhancing efficiency"
#     OTHER = "a goal not elsewhere classified, covering miscellaneous objectives that don't fit into standard categories"
#     PRODUCT = "a product goal, aimed at developing, improving, or managing a product or product line"
#     SALES = "a revenue goal, centered around increasing sales, expanding market share, and achieving financial targets"
#     STRATEGY = "a strategy goal, focused on long-term planning, setting organizational direction, and defining strategic priorities"
#     SUPPORT = "a customer support goal, aimed at improving customer service, resolving issues, and enhancing customer satisfaction"
#     TECHNOLOGY = "a technology or engineering goal, focused on technological advancements, software development, and technical infrastructure"

class Labels(str, enum.Enum):
    """Enumeration for text classification labels"""
    
    BUSINESS = "a business goal, focused on sales, marketing, and strategic planning, including increasing sales, market share, brand awareness, and defining strategic priorities"
    PEOPLE = "a people goal, focused on employee management, recruitment, development, retention, compliance with laws, regulations, contracts, and mitigating legal risks"
    PRODUCT = "a product goal, aimed at developing, improving, managing products, technological advancements, software development, and technical infrastructure"
    OPERATIONS = "an operations goal, focused on optimizing internal processes, supply chain management, enhancing efficiency, customer service, and resolving issues"
    OTHER = "a goal not elsewhere classified, covering miscellaneous objectives that don't fit into standard categories"



# base classification model

class SinglePrediction(BaseModel):
    """
    Class for a single class label prediction.
    """

    class_label: Labels

# classify function

def classify(data: str) -> SinglePrediction:
    """Perform single-class-label classification on the input text."""
    return client.chat.completions.create(
        model=GPT_MODEL_TEXT_ALIAS,
        response_model=SinglePrediction,
        max_retries=5,
        messages=[
            {
                "role": "user",
                # "content": f"Classify the following text: {data}",
                "content": f"""
                Correctly apply the most relevant enum to the provided {data}.
                Each enum represents a specific business category.
                Ensure that your classification is based on the primary focus and objectives of the data.
                """,
            },
        ],
    )  # type: ignore

# output with Single Prediction
for number, goal in enumerate(completion.goals, start=1):
    result = classify(goal)
    print((f"Goal {number}: {goal.sentence}\nClassification: {result.class_label.name}\nTotal tokens: {result._raw_response.usage.total_tokens}\n"))

In [ ]:
# multiclass prediction model

# multiclass labels

class Labels(str, enum.Enum):
    """Enumeration for text classification labels for a multiclass model"""

    HR = "a human resources goal, focused on employee management, recruitment, development, and retention"
    LEGAL = "a legal goal, aimed at ensuring compliance with laws, regulations, and contracts, and mitigating legal risks"
    MARKETING = "a marketing goal, centered around promoting products or services, increasing brand awareness, and generating leads"
    OPERATIONS = "an operations goal, focused on optimizing internal processes, supply chain management, and enhancing efficiency"
    OTHER = "a goal not elsewhere classified, covering miscellaneous objectives that don't fit into standard categories"
    PRODUCT = "a product goal, aimed at developing, improving, or managing a product or product line"
    SALES = "a revenue goal, centered around increasing sales, expanding market share, and achieving financial targets"
    STRATEGY = "a strategy goal, focused on long-term planning, setting organizational direction, and defining strategic priorities"
    SUPPORT = "a customer support goal, aimed at improving customer service, resolving issues, and enhancing customer satisfaction"
    TECHNOLOGY = "a technology or engineering goal, focused on technological advancements, software development, and technical infrastructure"


# multi class base model

class MultiPrediction(BaseModel):
    """
    Class for a multi-class enum prediction of labels.
    """

    # multi_class_label: List[Labels] = Field(...,description="Correctly apply one or more enums to the provided data. Never apply all enums")
    multi_class_label: List[Labels] = Field(...,description="Correctly apply one to three of the most relevant enums to the provided data. Never apply all enums.")

# multiclassify prompt
    
# multi_prompt = f"You are an expert data scientist and domain expert. Correctly apply one to three of the most relevant enums to the provided data. Never apply all enums."

multi_prompt = """
    You are a venture capital partner.
    Correctly apply one to three of the most relevant enums to the provided data.
    Each enum represents a specific business category.
    Ensure that your classification is based on the primary focus and objectives of the data.
    Do not apply all enums; only select those that are most applicable.
"""

# multiclassify function

def multiclassify(data: str) -> MultiPrediction:
    """Perform multi-class-label classification on the input text."""
    return client.chat.completions.create(
        model=GPT_MODEL_TEXT_ALIAS,
        response_model=MultiPrediction,
        max_retries=5,
        messages=[
            {
                "role": "system",
                "content": multi_prompt,
            },
        ],
    )  # type: ignore

# output with multiple classifications
for number, goal in enumerate(completion.goals, start=1):
    result = multiclassify(goal)
    label_names = [label.name for label in result.multi_class_label]
    label_names = ', '.join(label_names)
    print((f"Goal {number}: {goal.sentence}\nMultiClassification: {label_names}\nTotal tokens: {result._raw_response.usage.total_tokens}\n"))


## RANDOM STUFF

In [ ]:
print(result._raw_response.usage)

In [ ]:
print(completion._raw_response.usage)

TRYING OUT MULTICLASS

In [ ]:
# Try this version out using llm validator : https://www.linkedin.com/pulse/development-soql-generator-applying-rag-salesforce-data-toogood-22gre/?utm_source=rss&utm_campaign=articles_sitemaps

In [ ]:
messages = []
previous_response = freewriting.choices[0].message.content
next_message = free

messages.append({"role": "assistant", "content": previous_response})
messages.append({"role": "user", "content": next_message})

freewriting = client.chat.completions.create(
        model=GPT_MODEL_TEXT_ALIAS,
        messages=messages
    )

In [ ]:
print(freewriting.choices[0].message.content)

In [ ]:
previous_response = freewriting.choices[0].message.content
messages.append({"role": "assistant", "content": previous_response})

In [ ]:
len(messages)